In [ ]:
import numpy as np
from cvxopt import matrix, solvers
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge

Regularized Linear Regression (7-10)

In [ ]:
def load_data(file_name):
  data = []
  with open(file_name, 'r') as in_file:
    for line in in_file:
      items = line.strip().split()
      digit = int(float(items[0]))
      intensity = float(items[1])
      symmetry = float(items[2])
      data.append([digit, intensity, symmetry])

  return np.array(data)

def reg_lin_reg(Z, Y, lambd):
  N, d = Z.shape
  reg_mat = lambd * np.identity(d)
  w = np.linalg.inv(np.dot(Z.T, Z) + reg_mat).dot(np.dot(Z.T, Y))
  return w

def get_error(Z, y, w):
  predictions = np.sign(np.dot(Z, w))
  e_in = np.mean(predictions != y)
  return e_in

def q7():
  train_data = load_data("features.train")
  trainX = train_data[:, 1:]
  N = trainX.shape[0]
  trainZ = np.hstack((np.ones((N, 1)), trainX))

  digits = range(5, 10)
  lambd = 1

  for digit in digits:
    trainY = np.where(train_data[:, 0] == digit, 1, -1)
    w = reg_lin_reg(trainZ, trainY, lambd)
    print(f"{digit} vs all E_in: {get_error(trainZ, trainY, w)}")

print("Question 7")
q7()

Question 7
5 vs all E_in: 0.07625840076807022
6 vs all E_in: 0.09107118365107666
7 vs all E_in: 0.08846523110684405
8 vs all E_in: 0.07433822520916199
9 vs all E_in: 0.08832807570977919


In [ ]:
def transform(X):
  x_1 = X[:, 0]
  x_2 = X[:, 1]
  ones = np.ones_like(x_1)
  Z = np.column_stack([ones, x_1, x_2, x_1 * x_2, x_1**2, x_2**2])
  return Z

def q8():
  train_data = load_data("features.train")
  test_data = load_data("features.test")

  trainZ = transform(train_data[:, 1:])
  testZ = transform(test_data[:, 1:])

  digits = range(0, 5)
  lambd = 1

  for digit in digits:
    trainY = np.where(train_data[:, 0] == digit, 1, -1)
    testY = np.where(test_data[:, 0] == digit, 1, -1)
    w = reg_lin_reg(trainZ, trainY, lambd)
    print(f"{digit} vs all E_out: {get_error(testZ, testY, w)}")

print("Question 8")
q8()

Question 8
0 vs all E_out: 0.10662680617837568
1 vs all E_out: 0.02192326856003986
2 vs all E_out: 0.09865470852017937
3 vs all E_out: 0.08271051320378675
4 vs all E_out: 0.09965122072745392


In [ ]:
def q9():
  train_data = load_data("features.train")
  test_data = load_data("features.test")

  trainX = train_data[:, 1:]
  N = trainX.shape[0]
  trainZ = np.hstack((np.ones((N, 1)), trainX))
  trainZ_transform = transform(trainX)

  testX = test_data[:, 1:]
  N_test = testX.shape[0]
  testZ = np.hstack((np.ones((N_test, 1)), testX))
  testZ_transform = transform(testX)

  digits = range(0, 10)
  lambd = 1

  for digit in digits:
    trainY = np.where(train_data[:, 0] == digit, 1, -1)
    testY = np.where(test_data[:, 0] == digit, 1, -1)
    w = reg_lin_reg(trainZ, trainY, lambd)
    w_transform = reg_lin_reg(trainZ_transform, trainY, lambd)
    print(f"{digit} vs all E_in: {get_error(trainZ, trainY, w)}")
    print(f"{digit} vs all E_in (transform): {get_error(trainZ_transform, trainY, w_transform)}")
    print(f"{digit} vs all E_out: {get_error(testZ, testY, w)}")
    print(f"{digit} vs all E_out (transform): {get_error(testZ_transform, testY, w_transform)}\n")

print("Question 9")
q9()

Question 9
0 vs all E_in: 0.10931285146070498
0 vs all E_in (transform): 0.10231792621039638
0 vs all E_out: 0.11509715994020926
0 vs all E_out (transform): 0.10662680617837568

1 vs all E_in: 0.01522424907420107
1 vs all E_in (transform): 0.012343985735838706
1 vs all E_out: 0.02242152466367713
1 vs all E_out (transform): 0.02192326856003986

2 vs all E_in: 0.10026059525442327
2 vs all E_in (transform): 0.10026059525442327
2 vs all E_out: 0.09865470852017937
2 vs all E_out (transform): 0.09865470852017937

3 vs all E_in: 0.09024825126868742
3 vs all E_in (transform): 0.09024825126868742
3 vs all E_out: 0.08271051320378675
3 vs all E_out (transform): 0.08271051320378675

4 vs all E_in: 0.08942531888629818
4 vs all E_in (transform): 0.08942531888629818
4 vs all E_out: 0.09965122072745392
4 vs all E_out (transform): 0.09965122072745392

5 vs all E_in: 0.07625840076807022
5 vs all E_in (transform): 0.07625840076807022
5 vs all E_out: 0.07972097658196313
5 vs all E_out (transform): 0.07922

In [ ]:
def filter_data_15(data):
  X = data[:, 1:]
  y = data[:, 0]
  filteredX = X[(y == 1) | (y == 5)]
  filteredY = y[(y == 1) | (y == 5)]
  binY = np.where(filteredY == 1, 1, -1)
  return transform(filteredX), binY

def q10():
    train_data = load_data("features.train")
    test_data = load_data("features.test")

    trainZ, trainY = filter_data_15(train_data)
    testZ, testY = filter_data_15(test_data)

    lambdas = [0.01, 1]

    for lambd in lambdas:
        w = reg_lin_reg(trainZ, trainY, lambd)
        e_in = get_error(trainZ, trainY, w)
        e_out = get_error(testZ, testY, w)

        print(f"Lambda = {lambd}:")
        print(f"E_in: {e_in}")
        print(f"E_out: {e_out}\n")

q10()

Lambda = 0.01:
E_in: 0.004484304932735426
E_out: 0.02830188679245283

Lambda = 1:
E_in: 0.005124919923126201
E_out: 0.025943396226415096



SVM 11-12

In [ ]:
data = np.array([
    [1, 0, -1],
    [0, 1, -1],
    [0, -1, -1],
    [-1, 0, 1],
    [0, 2, 1],
    [0, -2, 1],
    [-2, 0, 1]
])

X = data[:, :2]
y = data[:, 2]
N = len(y)

K = np.zeros((N, N))
for i in range(N):
  for j in range(N):
      K[i, j] = (1 + np.dot(X[i], X[j])) ** 2

P = matrix(np.outer(y, y) * K)
q = matrix(-np.ones(N))
G = matrix(-np.eye(N))
h = matrix(np.zeros(N))
A = matrix(y, (1, N), 'd')
b = matrix(0.0)

solvers.options["show_progress"] = False
solution = solvers.qp(P, q, G, h, A, b)
w = np.array(solution['x']).flatten()

threshold = 1e-4
support_vectors = np.where(w > threshold)[0]

print(f"Number of support vectors: {len(support_vectors)}")

Number of support vectors: 5


RBF 13-18

In [ ]:
def target_function(X):
    x1 = X[:, 0]
    x2 = X[:, 1]
    return np.sign(x2 - x1 + 0.25 * np.sin(np.pi * x1))

def q13():
  gamma = 1.5
  N = 100
  runs = 1000
  not_separable_count = 0

  for run in range(runs):
      X = np.random.uniform(-1, 1, size=(N, 2))
      y = target_function(X)

      svm = SVC(kernel="rbf", gamma=10, C=1e3)
      svm.fit(X, y)
      y_pred = svm.predict(X)
      e_in = np.mean(y_pred != y)

      if e_in > 0:
          not_separable_count += 1

  percent_not_separable = (not_separable_count / runs) * 100
  print("Question 13")
  print(f"Percent not seperable: {percent_not_separable}")

q13()

Question 13
Percent not seperable: 0.2


In [ ]:
def lloyd_kmeans(X, K):
  kmeans = KMeans(n_clusters=K, n_init=10, max_iter=300)
  kmeans.fit(X)
  centers = kmeans.cluster_centers_
  labels = kmeans.labels_
  return centers, labels

def get_rbf_features(X, centers, gamma):
  Z = np.exp(-gamma * np.linalg.norm(X[:, np.newaxis] - centers, axis=2) ** 2)
  return Z

def train_linear_model(Z, y):
  Z_with_bias = np.hstack((np.ones((Z.shape[0], 1)), Z))
  ridge = Ridge(alpha=1e-8, fit_intercept=False)
  ridge.fit(Z_with_bias, y)
  w = ridge.coef_
  return w

def train_kernel_svm(X, y, gamma):
  svm = SVC(kernel="rbf", gamma=gamma, C=1e10)
  svm.fit(X, y)
  if svm.score(X, y) < 1.0:
      return None
  return svm

def error(model, model_type, testX, testY, centers=None, gamma=None):
  if model_type == "regular":
      testZ = get_rbf_features(testX, centers, gamma)
      temp_testZ = np.hstack((np.ones((testZ.shape[0], 1)), testZ))
      predictions = np.sign(np.dot(temp_testZ, model))
  elif model_type == "kernel":
      predictions = model.predict(testX)

  err = np.mean(predictions != testY)
  return err

def q14():
  num_runs = 1000
  N_train = 100
  N_test = 1000
  K = 9
  gamma = 1.5

  num_valid_runs = 0
  num_kernel_better = 0

  for run in range(num_runs):
    trainX = np.random.uniform(-1, 1, (N_train, 2))
    trainY = target_function(trainX)
    testX = np.random.uniform(-1, 1, (N_test, 2))
    testY = target_function(testX)

    centers, labels = lloyd_kmeans(trainX, K)
    if len(np.unique(labels)) < K:
      continue

    trainZ = get_rbf_features(trainX, centers, gamma)
    w = train_linear_model(trainZ, trainY)
    e_out_reg = error(w, "regular", testX, testY, centers, gamma)

    svm = train_kernel_svm(trainX, trainY, gamma)
    if svm is None:
      continue

    e_out_kernel = error(svm, "kernel", testX, testY)

    if e_out_kernel < e_out_reg:
      num_kernel_better += 1
    num_valid_runs += 1

  percent_kernel_better = (num_kernel_better / num_valid_runs) * 100
  print("Question 14")
  print(f"Percent runs where Kernel RBF better: {percent_kernel_better:}%")

q14()

Question 14
Percent runs where Kernel RBF better: 91.5%


In [ ]:
def q15():
  num_runs = 1000
  N_train = 100
  N_test = 1000
  K = 12
  gamma = 1.5

  num_valid_runs = 0
  num_kernel_better = 0

  for run in range(num_runs):
    trainX = np.random.uniform(-1, 1, (N_train, 2))
    trainY = target_function(trainX)
    testX = np.random.uniform(-1, 1, (N_test, 2))
    testY = target_function(testX)

    centers, labels = lloyd_kmeans(trainX, K)
    if len(np.unique(labels)) < K:
        continue

    trainZ = get_rbf_features(trainX, centers, gamma)
    w = train_linear_model(trainZ, trainY)
    e_out_reg = error(w, "regular", testX, testY, centers, gamma)

    svm = train_kernel_svm(trainX, trainY, gamma)
    if svm is None:
      continue

    e_out_kernel = error(svm, "kernel", testX, testY)

    if e_out_kernel < e_out_reg:
      num_kernel_better += 1
    num_valid_runs += 1

  percent_kernel_better = (num_kernel_better / num_valid_runs) * 100
  print("Question 15")
  print(f"Percent runs where Kernel RBF better: {percent_kernel_better}%")

q15()

Question 15
Percent runs where Kernel RBF better: 83.3%


In [ ]:
def q16(K):
  num_runs = 1000
  N_train = 100
  N_test = 1000
  gamma = 1.5

  num_valid_runs = 0
  num_kernel_better = 0
  total_ein = 0
  total_eout = 0

  print(f"K value {K}")

  for run in range(num_runs):
    trainX = np.random.uniform(-1, 1, (N_train, 2))
    trainY = target_function(trainX)
    testX = np.random.uniform(-1, 1, (N_test, 2))
    testY = target_function(testX)

    centers, labels = lloyd_kmeans(trainX, K)
    if len(np.unique(labels)) < K:
        continue

    trainZ = get_rbf_features(trainX, centers, gamma)
    w = train_linear_model(trainZ, trainY)
    total_ein += error(w, "regular", trainX, trainY, centers, gamma)
    total_eout += error(w, "regular", testX, testY, centers, gamma)

  print(f"e_in: {total_ein / num_runs}")
  print(f"e_out: {total_eout / num_runs}")

print("Question 16")
q16(9)
q16(12)

Question 16
K value 9
e_in: 0.03532000000000003
e_out: 0.05806699999999991
K value 12
e_in: 0.021779999999999945
e_out: 0.04642399999999993


In [ ]:
def q17(gamma):
  num_runs = 1000
  N_train = 100
  N_test = 1000
  K = 9

  num_valid_runs = 0
  num_kernel_better = 0
  total_ein = 0
  total_eout = 0

  print(f"gamma value {gamma}")

  for run in range(num_runs):
    trainX = np.random.uniform(-1, 1, (N_train, 2))
    trainY = target_function(trainX)
    testX = np.random.uniform(-1, 1, (N_test, 2))
    testY = target_function(testX)

    centers, labels = lloyd_kmeans(trainX, K)
    if len(np.unique(labels)) < K:
        continue

    trainZ = get_rbf_features(trainX, centers, gamma)
    w = train_linear_model(trainZ, trainY)
    total_ein += error(w, "regular", trainX, trainY, centers, gamma)
    total_eout += error(w, "regular", testX, testY, centers, gamma)

  print(f"e_in: {total_ein / num_runs}")
  print(f"e_out: {total_eout / num_runs}")

print("Question 17")
q17(1.5)
q17(2)

Question 17
gamma value 1.5
e_in: 0.03408000000000002
e_out: 0.05919599999999992
gamma value 2
e_in: 0.03903999999999997
e_out: 0.06334500000000007


In [ ]:
def q18():
  num_runs = 1000
  N_train = 100
  N_test = 1000
  K = 9
  gamma = 1.5

  num_valid_runs = 0
  num_kernel_better = 0
  zero_ein = 0

  for run in range(num_runs):
    trainX = np.random.uniform(-1, 1, (N_train, 2))
    trainY = target_function(trainX)
    testX = np.random.uniform(-1, 1, (N_test, 2))
    testY = target_function(testX)

    centers, labels = lloyd_kmeans(trainX, K)
    if len(np.unique(labels)) < K:
        continue

    trainZ = get_rbf_features(trainX, centers, gamma)
    w = train_linear_model(trainZ, trainY)
    e_in =  error(w, "regular", trainX, trainY, centers, gamma)

    if e_in == 0:
      zero_ein += 1

  print(f"Percent time e_in is 0: {(zero_ein / num_runs) * 100}")

print("Question 18")
q18()

Question 18
Percent time e_in is 0: 1.7000000000000002
